In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import scipy.sparse

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset_kompas_summarized.xlsx to dataset_kompas_summarized.xlsx


In [ ]:
df = pd.read_excel('dataset_kompas_summarized.xlsx')

In [ ]:
df

,index,title,raw timestamp,original,tags,author,url,cleaned,label,timestamp,cleaned token length,summarized
0,0,"Efek Ekor Jas Pencalonan Anies, Elektabilitas ...","21 Februari 2023, 15:30 WIB",Hasil jajak pendapat yang diselenggarakan Litb...,Survei Litbang Kompas;Elektabilitas Nasdem Nai...,NaN,https://video.kompas.com/watch/258152/efek-eko...,"Efek Ekor Jas Pencalonan Anies, Elektabilitas ...",0,2023-02-21 15:30:00,118,Hasil jajak pendapat yang diselenggarakan Litb...
1,1,"Ekonomi 2024 Ditargetkan Tumbuh 5,7 Persen, pa...","Kompas.com - 21/02/2023, 14:22 WIB","JAKARTA, KOMPAS.com - Pemerintah menargetkan p...",Jakarta;Ekonomi 2024,Penulis Yohana Artha Uly | Editor Aprillia Ika,http://money.kompas.com/read/2023/02/21/142238...,"Ekonomi 2024 Ditargetkan Tumbuh 5,7 Persen, pa...",0,2023-02-21 14:22:00,423,Pemerintah menargetkan pertumbuhan ekonomi di ...
2,2,"Survei Litbang Kompas: PDI-P, Gerindra, dan Go...","21 Februari 2023, 09:58 WIB","PDI-Perjuangan, Partai Gerindra, dan Partai Go...",Pdip;Pdi Perjuangan;Gerindra;Golkar;Demokrat;N...,NaN,https://video.kompas.com/watch/257988/survei-l...,"Survei Litbang Kompas: PDI-P, Gerindra, dan Go...",0,2023-02-21 09:58:00,260,"PDI-P, Gerindra, dan Golkar menempati posisi t..."
3,3,"Survei Litbang ""Kompas"": Popularitas Golkar Te...","Kompas.com - 21/02/2023, 05:23 WIB","JAKARTA, KOMPAS.com - Survei Litbang Kompas Ja...",Litbang Kompas;survei kepemimpinan nasional;su...,Penulis Tatang Guritno | Editor Bagus Santosa,http://nasional.kompas.com/read/2023/02/21/052...,"Survei Litbang ""Kompas"": Popularitas Golkar Te...",0,2023-02-21 05:23:00,406,"Popularitas Golkar Tertinggi, Diikuti PDI-P, d..."
4,4,"""Endorsement"" dan Basa-basi Politik ala Jokowi...","Kompas.com - 21/02/2023, 05:20 WIB","JAKARTA, KOMPAS.com - Presiden Joko Widodo la...",capres pemilu 2024;jokowi dukung ganjar;jokowi...,Penulis Fitria Chusna Farisa | Editor Fitria C...,http://nasional.kompas.com/read/2023/02/21/052...,"""Endorsement"" dan Basa-basi Politik ala Jokowi...",0,2023-02-21 05:20:00,262,Basis massa pendukung Jokowi diprediksi tak ak...
...,...,...,...,...,...,...,...,...,...,...,...,...
4724,4745,Nurul Arifin Ajak Perempuan Terjun ke Dunia Po...,"Kompas.com - 21/04/2017, 16:41 WIB","JAKARTA, KOMPAS.com - Ketua DPP Partai Golkar ...",Kartini;Film Kartini;RA Kartini;Nurul Arifin,Penulis Ihsanuddin | EditorInggried Dwi Wedhas...,http://nasional.kompas.com/read/2017/04/21/164...,Nurul Arifin Ajak Perempuan Terjun ke Dunia Po...,0,2017-04-21 16:41:00,314,Ketua DPP Partai Golkar Nurul Arifin mendorong...
4725,4746,Kebangkitan Lawan Politik Jokowi...,"Kompas.com - 21/04/2017, 11:33 WIB","JAKARTA, KOMPAS.com - Pasangan Anies Baswedan-...",Pilpres 2019;Prabowo Subianto;Jokowi,Penulis Ihsanuddin | EditorSandro Gatra,http://nasional.kompas.com/read/2017/04/21/113...,Kebangkitan Lawan Politik Jokowi... Pasangan ...,0,2017-04-21 11:33:00,1027,Pasangan Anies Baswedan-Sandiaga Uno memenangi...
4726,4747,Ridwan Kamil Akui Pilkada DKI Berdampak pada S...,"Kompas.com - 20/04/2017, 16:45 WIB","BANDUNG, KOMPAS.com - Wali Kota Bandung Ridwan...",Ridwan Kamil;Pilkada DKI,"Penulis Kontributor Bandung, Dendi Ramdhani | ...",http://regional.kompas.com/read/2017/04/20/164...,Ridwan Kamil Akui Pilkada DKI Berdampak pada S...,0,2017-04-20 16:45:00,453,"Ridwan Kamil menilai, hasil Pilkada DKI Jakart..."
4727,4748,Anies Belum Berpikir Apakah Akan Menjadi Anggo...,"Kompas.com - 20/04/2017, 14:41 WIB","JAKARTA, KOMPAS.com - Calon gubernur DKI Jakar...",Anies Baswedan-Sandiaga Uno,Penulis David Oliver Purba | EditorEgidius Pat...,http://megapolitan.kompas.com/read/2017/04/20/...,Anies Belum Berpikir Apakah Akan Menjadi Anggo...,0,2017-04-20 14:41:00,241,Calon gubernur DKI Jakarta Anies Baswedan masi...


In [ ]:
# Mapping label menjadi angka
label_mapping = {'Real': 1, 'Fake': 0}
df['label'] = df['label'].map(label_mapping)

# Pastikan mapping berhasil
print(df['label'].value_counts())

Series([], Name: count, dtype: int64)


In [ ]:
X = df['title']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
label_encoder = LabelEncoder()

# Fit the encoder to your training labels
label_encoder.fit(y_train)


LabelEncoder()

In [ ]:
max_vocab_size = 10000
sequence_length = 100

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_size,  # Jumlah kata maksimal
    output_mode='int',          # Mengubah teks menjadi indeks
    output_sequence_length=sequence_length  # Panjang sequence yang seragam
)

# Adapt vectorizer ke data teks
vectorizer.adapt(X_train)

In [ ]:
# Mengubah teks menjadi numerik
X_train_vectorized= vectorizer(X_train)
X_test_vectorized = vectorizer(X_test)


In [ ]:
# Transform the training and testing labels
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Model Sequential dengan lapisan embedding dan LSTM
model = Sequential([
    Embedding(input_dim=max_vocab_size, output_dim=128, input_length=sequence_length),  # Embedding layer
    LSTM(64, return_sequences=True),  # LSTM Layer
    LSTM(32),                         # LSTM Layer kedua
    Dense(1, activation='sigmoid')    # Output layer untuk klasifikasi biner
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Melatih model
history = model.fit(
    X_train_vectorized.numpy(),
    y_train_encoded,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)
if scipy.sparse.issparse(X_test_vectorized):
  X_test_evaluated = X_test_vectorized.toarray()
else:
  X_test_evaluated = X_test_vectorized
# Check the shape of your input data and the input shape of the model
print("Shape of X_test_evaluated:", X_test_evaluated.shape)
print("Input shape of the model:", model.input_shape)
# Convert X_test_vectorized and y_test to TensorFlow tensors
X_test_tensor = tf.convert_to_tensor(X_test_vectorized.numpy())
y_test_tensor = tf.convert_to_tensor(y_test_encoded)
# Evaluasi model
loss, accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print(f"Model Accuracy: {accuracy*100:.2f}%")

Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 17s 123ms/step - accuracy: 0.9464 - loss: 0.1221 - val_accuracy: 1.0000 - val_loss: 7.3018e-04
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - accuracy: 1.0000 - loss: 6.1323e-04 - val_accuracy: 1.0000 - val_loss: 3.8678e-04
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 12s 131ms/step - accuracy: 1.0000 - loss: 3.5600e-04 - val_accuracy: 1.0000 - val_loss: 2.8496e-04
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 12s 131ms/step - accuracy: 1.0000 - loss: 2.6856e-04 - val_accuracy: 1.0000 - val_loss: 2.2651e-04
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 13s 136ms/step - accuracy: 1.0000 - loss: 2.1555e-04 - val_accuracy: 1.0000 - val_loss: 1.8636e-04
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 20s 131ms/step - accuracy: 1.0000 - loss: 1.7832e-04 - val_accuracy: 1.0000 - val_loss: 1.5641e-04
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 20s 130ms/step - accuracy: 1.0000 - loss: 1.5017e-04 - val_accuracy: 1.0000 - val_loss: 1.3282e-04
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 21s 131

In [ ]:
# Simpan model
model.save('saved_model.h5')

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 100, 64)             │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,025,573 (15.36 MB)

 Trainable params: 1,341,857 (5.12 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,683,716 (10.24 MB)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
model = load_model('saved_model.h5')

In [ ]:
def preprocess_text(news, tokenizer, max_length):
    sequence = tokenizer.texts_to_sequences([news])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

In [ ]:
def predict_news_tf(news, tokenizer, max_length):
    news_preprocessed = preprocess_text(news, tokenizer, max_length)
    probabilities = model.predict(news_preprocessed)
    predicted_class = 1 if probabilities[0] > 0.5 else 0
    return predicted_class, probabilities[0]

In [ ]:
tokenizer = Tokenizer(num_words=10000)
max_length = 100

In [ ]:
news_input = "Efek Ekor Jas Pencalonan Anies, Elektabilitas Nasdem Naik 3 Persen"
result, prob = predict_news_tf(news_input, tokenizer, max_length)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step


In [ ]:
label = "Real" if result == 1 else "Fake"
print(f"Berita: {news_input}\nHasil Prediksi: {label} (Probabilitas: {prob[0]})")

Berita: Efek Ekor Jas Pencalonan Anies, Elektabilitas Nasdem Naik 3 Persen
Hasil Prediksi: Fake (Probabilitas: 8.160222932929173e-05)
